# Spark tutorial

## Setup
Import the `.ipynb` file into Jupyter and open this as usual.  MAKE SURE YOU ARE ON THE VPN IF YOU USE YOUR LAPTOP!

First, we will install a package which will allow us to access Spark with the Python API: `findspark`.  Execute the code in the cell below to install `findspark`.


In [ ]:
%%bash
pip install findspark

With this new package, when we run `findspark.init()`, it creates a Pyspark environment we can run Spark applications in, as follows.

In [ ]:
import findspark
findspark.init('/home/comp6235/spark-2.4.0-bin-hadoop2.7')

import pyspark
from pyspark import SparkContext, SparkConf


We created a `SparkContext` above, which represents a connecting to a computing cluster.  If you start the PySprak shell, it will be created automatically as `sc`, but here we need to create it manually.

With the `SparkContext` object, we can create RDDs.  It is also possible to configure the `SparkContext` with a `SparkConf` object passed as a  paprameter:

In [ ]:
conf = SparkConf().setMaster('local').setAppName('My application')

sc = SparkContext(conf=conf)
sc

The output of `sc` gives you a link to Spark UI of your context.  You can use this to visualise the jobs you're running.

Note that you can only create one of these `SparkContext` objects at a time.  If you wish to run it again, you will need to call `sc.stop()` to get rid of the old one and to start it running again.

With our `SparkContext`, we can now start creating an RDD.  There are two ways of creating an RDD: through an internal object or class, or an external dataset.  For this, we are going to use a text file containing the text of the complete Sherlock Holmes stories.  I have left a copy of the text at http://edshare.soton.ac.uk/19180/1/holmes_complete.txt.  Download a copy of ths file into the same directory as your tutorial.

We call the function `sc.textFile`, and this returns an RDD:

In [ ]:
rdd = sc.textFile('holmes_complete.txt')
rdd

A partition is a logical chunk of a large distributed data set.  This represents the amount of parallel tasks you are going to be using.  By default it goes to one per core, but you can set it by adding a parameter to the RDD object you created.  We can see the amount of partitions we have by running the cell below:

In [ ]:
rdd.getNumPartitions()

## Actions and transformations

You can see the output refers to an RDD object, but nothing has happened yet, because the RDD uses lazy processing and does not do anything until it has to.  By contrast, if we apply an action like `count()` or `reduce()` then it will return an object rather than an RDD which is illustrative that some processing has occurred.  We will call an action `count()` here which returns the amount of lines in the RDD:

In [ ]:
rdd.count()

We can see how the transformation returns an RDD by executing the next cell.  In this we are using the `filter()` transformation.  The filter is using Python `lambda` expressions to identify any instances of the name "_Holmes_" inside the text file.  A lambda expression is like an anonymous function, it would also be possible to define a function and pass that as a parameter instead.

In [ ]:
holmes = rdd.filter(lambda line: "Holmes" in line)
print(holmes.count())

If you need a bit more of an introduction to lambda functions, check out http://www.diveintopython.net/power_of_introspection/lambda_functions.html.  These are very useful in Spark, so it's worth knowing how they work.  In the next cell, try and get the equivalent outcome by defining a function and passing it as a parameter to the `filter` function.  Call the output `holmes_2`.

In [ ]:
# YOUR CODE HERE


This returns a new RDD object, and does not mutate the initial RDD object either - despite referring to a filter operation on it.  The following will return the first line of the document (which includes the name "Holmes".  Note that it will not operate on `rdd`, but rather will operate on `holmes`, as can be seen from executing the following cell.

Look at the output below, and notice how the amount of lines on the `rdd` object remains the same as above in contrast to the amount of lines in `holmes`.

In [ ]:
print(rdd.count())
print(holmes.count())

If we wish, we can perform set operations on the data.  We will demonstrate this by creating another RDD object, containing only lines which have the name "_Watson_" in them, and then get the intersection and union of the two.

Create the `watson` RDD object, and then call the `first()` action to return the first line, and then call the `count()` to see how many lines there are in the book which satisfy that filter.

In [ ]:
# YOUR CODE HERE
# New RDD and show the first line here


In [ ]:
# YOUR CODE HERE
# How many lines in the Watson RDD?


We can create a new RDD by calling the intersection or the union of the two:

In [ ]:
holmes_and_watson = holmes.intersection(watson)
holmes_and_watson

In [ ]:
holmes_and_watson.count()

Try and find the union of lines with the name "Holmes" and the name "Watson":

In [ ]:
# YOUR CODE HERE


In [ ]:
# YOUR CODE HERE
# How many lines in the file?


## Lineage

We discussed lineage: the process by which RDDs are connected.  By creating these different RDDs, we are able to get an idea of the lineage of the We can get an idea of the lineage of an RDD object by calling the function `toDebugString()`.  Call this on the union of the `holmes` and `watson` RDDS:

In [ ]:
holmes_or_watson.toDebugString()

Alternatively, there is a UI listening on port `4040` as part of Spark (see above where we printed the output of `sc`).  If you open that in a new tab, and click on the "jobs" you can click through to see the visualisation of the DAG which is how the lineage is represented.

## MapReduce

Spark started off as a batch processing library, although one more efficient than Hadoop, and as such it is possible to run MapReduce tasks.  This is done on the RDD object through the `map()` and `flatMap()` functions, (which are transformations) and the `reduce` function which is an action.

Both `map()` and `flatMap()` transformation take a function as an argument, and applies it to each element in the RDD.  `map()` returns a new RDD the same size as the original RDD, whereas `flatMap()` transforms the RDD of length _n_ into _n_ collections, and flattens these into a single RDD of results.  See https://stackoverflow.com/questions/22350722/what-is-the-difference-between-map-and-flatmap-and-a-good-use-case-for-each for a good explanation.

The typical "Hello, world" example, as last week, is that of word count.  We will do a word count on the text we have in the `holmes_complete.txt` file, starting with the `rdd` variable we already have.

It's just text this time, so we don't have to parse through a CSV file which should make it a bit easier.  We will start with applying the `flatMap` to get the words tokenised.  The `take(n)` function gives us a list of the first `n` items in the RDD which can be useful for debugging.

In [ ]:
word_split = rdd.flatMap(lambda line: line.split(' '))
word_split.take(10)

Using `map` on `word_split`, create a new RDD which outputs the words as a series of tuples of length two, each containing the word and the number 1.  Then, output the first 30 pairs.

In [ ]:
# YOUR CODE HERE


Now we have the mapped part of the MapReduce function, the next step is to get the `reduce()` action.  More specifically, we need to run the `reduceByKey()`.  From the documentation:
<blockquote>
When called on a dataset of <code>(K, V)</code> pairs, returns a dataset of <code>(K, V)</code> pairs where the values for each key are aggregated using the given reduce function func, which must be of type <code>(K, V) => V</code>. Like in </code>groupByKey</code>, the number of reduce tasks is configurable through an optional second argument. 
</blockquote>

In [ ]:
reduced_words = word_count_1.reduceByKey(lambda a,b: a + b) 
reduced_words.take(10)

Although we can see that there is a wordcount, it would be more useful for it to be done in order.  Here, we use the `sortBy` function, which takes a function and operates on the second part of the tuple (the value) in order to sort in descending order by amount of occurrences.

In [ ]:
reduced_words.sortBy(lambda a: -a[1]).take(10)

## Extra challenge

* You will notice that there are many common words in the output which are not very useful for analysis.  Try and figure out a way of getting rid of these words from the output.  They are known as "stop words"
* Clear the whitespace from the document

### Streaming

If you are interested in seeing how the streaming works, check out the following link for streaming on Twitter data https://www.linkedin.com/pulse/apache-spark-streaming-twitter-python-laurent-weichberger